In [20]:
import requests
import os
import json
from google.cloud import storage
from dataverk_vault import api as vault_api

In [21]:
res = requests.get("https://data.ssb.no/api/klass/v1/versions/33")

In [22]:
vault_api.set_secrets_as_envs()

In [23]:
with open(f"creds.json", "w") as f:
    f.write(os.environ["STYRK_BUCKET_CREDS"])
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = f"creds.json"

In [24]:
client = storage.Client()

In [25]:
bucket = client.get_bucket("styrk-bucket")

In [26]:
blob = bucket.blob("styrk-koder/styrk.json")

In [27]:
blob.upload_from_string(json.dumps(res.json()))

## Convert to csv

In [28]:
import pandas as pd
df = pd.DataFrame.from_records(res.json()["classificationItems"])
df.to_csv("styrk.csv", index = None)#, sep=";")

In [29]:
blob = bucket.blob("styrk-koder/styrk.csv")

In [30]:
with open("styrk.csv", "rb") as my_file:
    blob.content_encoding = "utf-8"
    blob.upload_from_file(my_file)

In [31]:
blob = bucket.blob("styrk-koder/styrk.csv")

In [32]:
with open(f"../styrk/data/test.csv", "wb") as f:
    blob.download_to_file(f)